In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()

# **Restructuring the dataset in a new folder with the csv above and the images**

In [2]:
import os
import shutil
import pandas as pd

def extract_info_from_hea(folder_path, label_mapping):
    data = []

    # Iterate over all files in the given folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".hea"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                lines = file.readlines()
                image_path = ""
                labels = ""
                
                # Iterate over each line in the .hea file
                for line in lines:
                    if 'Labels' in line:
                        labels = line.split(":")[1].strip()
                    if 'png' in line:
                        image_path = line.split(":")[1].strip()
                
                # Create a dictionary to hold the image path and label information
                label_info = {label: 0 for label in label_mapping}
                if labels:
                    for label in labels.split(","):
                        if label.strip() in label_mapping:
                            label_info[label.strip()] = 1
                
                label_info["Image_Name"] = image_path
                data.append(label_info)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df

def move_files_and_create_structure(src_folder, dest_folder, df):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    
    # Create the images subfolder
    images_folder = os.path.join(dest_folder, 'images')
    if not os.path.exists(images_folder):
        os.makedirs(images_folder)
    
    # Copy the images
    for index, row in df.iterrows():
        src_image_path = os.path.join(src_folder, row['Image_Name'])
        dest_image_path = os.path.join(images_folder, row['Image_Name'])
        if os.path.exists(src_image_path):
            shutil.copy2(src_image_path, dest_image_path)
    
    # Save the DataFrame to a CSV file
    csv_path = os.path.join(dest_folder, 'multilabel_classification.csv')
    df.to_csv(csv_path, index=False)

# Define the label mapping
label_mapping = {"NORM", "Acute MI", "Old MI", "STTC", "CD", "HYP", "PAC", "PVC", "AFIB/AFL", "TACHY", "BRADY"}

# Set the folder paths
src_folder = '//kaggle/input/physionet-flash-data/data/records500/01000'
dest_folder = '/kaggle/working/new-data'

# Extract information from the .hea files and create the DataFrame
df = extract_info_from_hea(src_folder, label_mapping)

# Move the files and create the new folder structure
move_files_and_create_structure(src_folder, dest_folder, df)


In [3]:
len(os.listdir("/kaggle/working/new-data/images"))

1000

# **Prepare the "physionet-100-00000"**


In [4]:
!git clone https://github.com/physionetchallenges/python-example-2024.git

Cloning into 'python-example-2024'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 116 (delta 45), reused 50 (delta 28), pack-reused 44
Receiving objects: 100% (116/116), 232.04 KiB | 6.82 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [5]:
!python "/kaggle/working/python-example-2024/prepare_image_data.py" -i "/kaggle/input/physionet-100-00000/content/data/ptb-103/records100/00000" -o "/kaggle/working/records100/00000"

In [6]:
import os
len(os.listdir("/kaggle/working/records100/00000"))

2134